In [1]:
import krpc
import time
import math
import numpy as np
import matplotlib.pyplot as plt

**Initialize kRPC Connection & Vessel**

In [2]:
# Once kRPC client is active in-game and at vehicle screen..
conn = krpc.connect(name='orbit_krpc_test', 
                    address='127.0.0.1',
                    rpc_port=50000,
                    stream_port=50001)

In [3]:
# Craft initialize        
orion = conn.space_center.active_vessel
gateway = conn.space_center.vessels[1]
print(conn.krpc.get_status().version)

0.4.8


**Orbital Mechanics Definitions**

In [4]:
def calc_burn_duration(vessel, dV):
    ''' Calculates the burn time for a given vessel and a given delta-V. '''
    m = vessel.mass
    isp = vessel.specific_impulse
    thrust = vessel.available_thrust
    term1 = (m*isp*9.81)/(thrust)
    term2 = (1-e**(dV/(isp*9.81)))
    return abs(term1*term2)


def vis_viva(r, a, M):
    '''
    Calculates the orbital speed of a vessel at some point in an
    elliptical orbit given the semi-major axis and mass of orbiting
    body.
    '''
    return np.sqrt(G*M*((2/r)-(1/a)))


def orbital_period(a, M):
    ''' 
    Calculates the orbital period of a satellite given
    the semi-major axis of the orbit and the mass of the
    orbiting body.
    '''
    mu = G*M
    return (2*np.pi)*np.sqrt(a**3/mu)


def v_pe_hyperbolic(M, a, e):
    ''' Find periapsis speed of hyperbolic orbit. '''
    k = G*M  # gravitational parameter
    return ((-k/a)*(1+e)/(e-1))**0.5

**Define Celestial Bodies**

In [5]:
kerbin = conn.space_center.bodies['Kerbin']
mun = conn.space_center.bodies['Mun']

**Reference Frame**

In [6]:
# Non-rotating Kerbin Centered Reference Frame
kerbin_nrrf = kerbin.non_rotating_reference_frame

**Properties of Bodies**

In [8]:
# mass in kg
massKerbin = kerbin.mass  
massMun = mun.mass

# equatorial radius in m
radKerbin = kerbin.equatorial_radius
radMun = mun.equatorial_radius

# gravitational parameter in m^3/s^2
muKerbin = kerbin.gravitational_parameter
muMun = mun.gravitational_parameter

# Sphere-of-Influence in m
munSOI = mun.sphere_of_influence

# Rotational Period + Velocity
T_m = mun.rotational_period  # period in sec
w_m = mun.rotational_speed  # sidereal speed in rad/s

# Orbital Period + Velocity
v_m = np.array([0, mun.orbit.orbital_speed, 0])  # i=0, j, k=0

print('Kerbin Radius =', radKerbin)
print('Mun Radius =', radMun)
print('Mun Orbital Velocity @ Patch Point =', v_m)
print('Mun Rotational Speed =', w_m)

G = 6.67408e-11  # (m^3/kg/s^2) gravitational constant


Kerbin Radius = 600000.0
Mun Radius = 200000.0
Mun Orbital Velocity @ Patch Point = [  0.        542.4942396   0.       ]
Mun Rotational Speed = 4.520785660133697e-05


**Trans-Munar Injection**

In [9]:
# Aiming for true anomaly at intercept = 180 deg 

print('Preparing for Trans-Munar Injection...')

# set Mun as target
conn.space_center.target_body = mun
time.sleep(1)

Preparing for Trans-Munar Injection...


In [22]:
# Arrival Angle @ Patch Point
munAA = 0 * np.pi/180  # mun arrival angle (deg->rad) - depends on patch point being location

# Mun relative to Kerbin 
r_m = mun.position(kerbin_nrrf)  # i, j=0, k
ang_pos_mun = math.atan(r_m[2]/r_m[0])*180/np.pi  # relative to DN = +x

# Position vector of Spacecraft relative to Mun @ patch point
r_2 = np.array([-munSOI*math.cos(munAA), 0, munSOI*math.sin(munAA)])  # i,j=0, k           
# Position vector of Spacecraft at Patch Point relative to Kerbin
r_1 = r_m + r_2  

# Spacecraft position @ Current
alpha_0 = 0  # angle from Kerbin-Mun line @ TMI
r_current = np.array(orion.orbit.position_at(conn.space_center.ut, kerbin_nrrf))  # i, j~=0, k

# Spacecraft position @ TMI
r_0 = np.array(orion.orbit.position_at(conn.space_center.ut + orion.orbit.time_to_periapsis, kerbin_nrrf))  # i, j~=0, k

# # *****!!!~!~!!~!~needs to be adjusted to be flexible
# a_transf = (mun.orbit.apoapsis + radKerbin + orion.orbit.periapsis) / 2  # semi-major axis of transfer orbit

print('UT =', conn.space_center.ut)
print('Time to Periapsis =', orion.orbit.time_to_periapsis, 'sec')  # sec

print('Mun currently at position:', r_m, '=', ang_pos_mun, 'deg')
print('Orion currently at:', r_current)

print('\nMun to Orion at Patch Point:', r_2)
print('Kerbin to Orion at Patch Point:', r_1)
print('Kerbin to Orion at TMI:', r_0)

UT = 331408.9629346091
Time to Periapsis = 1957.3066497433306 sec
Mun currently at position: (-6740723.877348406, 0.0, -9927871.957743267) = 55.82468507275559 deg
Orion currently at: [7.13861586e+03 1.22107533e+00 6.99766143e+05]

Mun to Orion at Patch Point: [-2429559.        0.        0.]
Kerbin to Orion at Patch Point: [-9170282.87734841        0.         -9927871.95774327]
Kerbin to Orion at TMI: [8.61183881e+03 1.22104524e+00 6.99749563e+05]


In [24]:
# Sweep Angle 
theta_tgt = math.atan(r_1[1]/r_1[0]) * 180/np.pi  # angle from local horizontal to patch point r_1
theta = 180 + theta_tgt  # true anomaly at r_1
theta_0 = 0  # true anomaly at r_0
sw_deg = theta - theta_0
sw_rad = sw_deg * np.pi/180

t_a = orion.orbit.true_anomaly*180/np.pi  # in deg

print('Sweep Angle =', sw_deg, 'deg', sw_rad, 'rad')
print('Current True Anomaly =', t_a, 'deg')

Sweep Angle = 180.0 deg 3.141592653589793 rad
Current True Anomaly = 6.956594753280709 deg


In [25]:
# Define Position Magnitudes
r0 = np.linalg.norm(r_0)
r1 = np.linalg.norm(r_1)
r2 = np.linalg.norm(r_2)

print(r0, r1, r2)

In [26]:
# Find Optimal Initial Flight Path Angle, start using fp_ang_i = 10
fp_ang_i = 0  # deg

h1 = math.sqrt(muKerbin*r0) * math.sqrt((1-math.cos(sw_rad)) / (r0/r1 + math.sin(sw_rad)*math.tan(fp_ang_i*np.pi/180) - math.cos(sw_rad))) 

print(h1)

2167834725.956549


In [31]:
f = 1 - (muKerbin*r1/h1**2) * (1 - math.cos(sw_rad))
g = r0*r1/h1 * math.sin(sw_rad)
g_dot = 1 - (muKerbin*r0/h1**2) * (1 - math.cos(sw_rad))

print('f =', f, 'g =', g, 'g_dot =', g_dot)

v_0 = (1 / g) * (r_1 - (f * r_0))
v_1 = (1 / g) * ((g_dot * r_1) - r_0)

print('Velocity @ TMI:', v_0)
print('Velocity @ Target:', v_1)

f = -19.312669827724935 g = 5.3429122889842e-13 g_dot = -0.05177947994349408
Velocity @ TMI: [-1.68521675e+19  4.41363106e+13  6.71199547e+18]
Velocity @ Target: [ 8.72596469e+17 -2.28535521e+12 -3.47543635e+17]


In [32]:
# Calculate TMI delta-V
vC = math.sqrt(muKerbin/r0)  # Circular orbit velocity
v0 = np.linalg.norm(v_0)
dv_tmi = math.sqrt(vC**2 + v0**2 - 2*vC*v0*math.cos(fp_ang_i*np.pi/180))

print('Required dV =', dv_tmi, 'm/s')

Required dV = 1.8139637050876369e+19 m/s


In [ ]:
# Calculate TMI delta-V
vC = math.sqrt(muKerbin/r0)  # Circular orbit velocity
v0 = np.linalg.norm(v_0)
dv_tmi = math.sqrt(vC**2 + v0**2 - 2*vC*v0*math.cos(fp_ang_i*np.pi/180))

print(dv_tmi)

In [ ]:
tmi = orion.control.add_node(ut=ut_tmi)
tmi.prograde = dv_tmi

In [ ]:
# set SAS to maneuver and wait until oriented
print('Orienting for maneuver.')

orion.control.sas_mode = conn.space_center.SASMode.maneuver
orion.control.rcs = True
time.sleep(3)

burn_time = calc_burn_duration(orion, dv_tmi)
print(burn_time)

In [ ]:
print('Burn baby, burn.')

# execute maneuver
while conn.space_center.ut < (ut_tmi - (burn_time / 2)):
    time.sleep(0.1)

while tmi.remaining_delta_v > 15:
    orion.control.throttle = 1
while tmi.remaining_delta_v > 0.1:
    orion.control.throttle = 0.05

tmi.remove()
orion.control.throttle = 0
orion.control.rcs = False

print('TMI complete.')

In [ ]:
tmi.remaining_delta_v